In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('../')
import utilities


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
tickers = pd.read_csv('../data_cleaning/stocks/dense_95.csv')

# for local testing
#tickers = tickers.head(5)

In [ ]:
tickers["ticker"].values

In [ ]:
# count the days in each stock

def unique_days(ticker):
    df = utilities.load_stock(ticker)
    days = utilities.count_days(df)

    return days

for i in range(len(tickers)):
    ticker = tickers["ticker"].values[i]
    days = unique_days(ticker)
    print(ticker, days)
    tickers.at[i, "data_days"] = days

In [ ]:
tickers.head()

In [ ]:
tickers.to_csv('selected_days.csv', index=False)